# A Multilevel Model

Assume a model of voting that predicts the ith respondents vote choice in the tth election as a function of their authoritarianism score, $x_{auth, i, t}$, and a set of control variables, $z_{k, i, t}$, where $k$ denotes the $k$th control variable. The model can be written as:

$pr(y_{i,t}| x_{auth, i, t}; x^2_{auth, i, t}; z_{k, i, t}) =\beta_{0,k} + \beta_{1,t} x_{auth, i, t} +  \beta_{2,t} x_{auth, i, t}^2 + \sum_{k=1}^K \beta_k z_{k,i,t} +  \epsilon_{t} +  \epsilon_{i,t}$

where $K$ denotes the number of control variables, $t$ cross-sectional year, and $i$, the $i$ th respondent (nested within the $t$ ANES survey). Wide priors were used for all the parameters, and the correlations between slopes and intercepts were always estimated (with the exception of the random intercept only model). The specification is general enough to allow for different error distributions.

In [ ]:
## These were loaded in the container build.
library(brms)
#library(tidyverse)
library(ggplot2)
library(modelr)
library(tidybayes)
library(dplyr)
library(cowplot)
data_location = "/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/"
#### Load cleaned data
load("/Users/Chris/Dropbox/github_repos/Authoritarianism_V2/clean_data/pooled.auth.rda")  ### Just work from this data; everything should be here, recoded.
##### persistent ggtheme
ggtheme = theme(
        plot.title=element_text(face="bold",hjust=-.08,vjust=0,colour="#3C3C3C",size=12),
        axis.text.x=element_text(size=9,colour="#535353",face="bold"),
        axis.text.y=element_text(size=9,colour="#535353",face="bold"),
        axis.title.y=element_text(size=11,colour="#535353",face="bold",vjust=1.5),
        axis.ticks=element_blank(),
        panel.grid.major=element_line(colour="#D0D0D0",size=.25),
        panel.background=element_rect(fill="white")) 
####
data$authoritarianism_2 = data$authoritarianism^2 # Attempted with orthogonal contrasts, but the incompatibility with other stuff made it a challenge

In [2]:
### I create smaller, more tractable versions of the data rather than operating on the full data frame, data.
tmp_dat = data[,c("vote", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 

This is the random intercept model. I ran it over 3000 iterations, 3 chains. 

In [ ]:
fit1 <- brm(vote~ female + age + college + income + jewish + 
                 catholic + other + authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "logit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 1000)